In [9]:
#instalare dependente
!pip install pandas numpy tensorflow prophet statsmodels
!pip install xgboost lightgbm


In [10]:
import pandas as pd
import numpy as np
import pickle
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber, MeanAbsoluteError
from sklearn.metrics import mean_absolute_error, mean_squared_error
from prophet import Prophet
import warnings
import xgboost as xgb
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation

# --- CONFIGURARE GENERALA ---
warnings.filterwarnings('ignore')
tf.random.set_seed(42)

print("--- START MODEL TRAINING  ---", flush=True)

# 1. INCARCARE DATE PROCESATE
print("Incarcare date...", end="", flush=True)
try:
    with open('processed_data.pkl', 'rb') as f:
        data = pickle.load(f)
    print(" GATA!", flush=True)
except FileNotFoundError:
    print("\n[EROARE] Nu gasesc fisierul 'processed_data.pkl'. Ruleaza intai pasii de preprocesare.")
    raise

# Extragere variabile din pachet
X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
test_data = data['test_data']
scaler = data['scaler'] 
df_1min = data['df_1min'] 
train_size = data['train_size']
WINDOW_SIZE = data['WINDOW_SIZE']

# Verificam dimensiunea input-ului (cate features avem?)

n_features = X_train.shape[2]
print(f"[INFO] Input Features: {n_features} (inclusiv Cluster pt regimuri)", flush=True)

# Dictionare rezultate
rezultate = {}
timpi = {}

# ======================================================
# MODEL 1: PROPHET (Ajustat cu Sezonalitate & Regresori)
# ======================================================
print("\n--- 1. PROPHET (Tuned) ---", flush=True)

# Pregatire dataset specific Prophet
# Prophet vrea coloane: 'ds' (timp), 'y' (target).
df_prophet = df_1min.reset_index()[['dt', 'Aggregate', 'Cluster', 'Hour_Sin', 'Hour_Cos', 'IsWeekend']]
df_prophet.columns = ['ds', 'y', 'Cluster', 'Hour_Sin', 'Hour_Cos', 'IsWeekend']

df_prophet_train = df_prophet.iloc[:train_size]
df_prophet_test = df_prophet.iloc[train_size:]

# Configurare CERINTA: daily_seasonality=True, weekly_seasonality=True
m_prophet = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    uncertainty_samples=0
)

# Adaugam informatiile extra (Regimuri + Timp)
m_prophet.add_regressor('Cluster')    
m_prophet.add_regressor('Hour_Sin')   # Info ciclica
m_prophet.add_regressor('IsWeekend')  # Info weekend

# Antrenare
start = time.time()
print("   -> Fit Prophet...", flush=True)
m_prophet.fit(df_prophet_train)
timpi['Prophet'] = time.time() - start

# Predictie
print("   -> Predictie Prophet...", flush=True)
future = m_prophet.make_future_dataframe(periods=len(df_prophet_test), freq='1min')
# Trebuie sa adaugam valorile regressorilor pentru viitor (le luam din test set)
future['Cluster'] = pd.concat([df_prophet_train['Cluster'], df_prophet_test['Cluster']]).values
future['Hour_Sin'] = pd.concat([df_prophet_train['Hour_Sin'], df_prophet_test['Hour_Sin']]).values
future['IsWeekend'] = pd.concat([df_prophet_train['IsWeekend'], df_prophet_test['IsWeekend']]).values

forecast = m_prophet.predict(future)
pred_prophet = forecast['yhat'].iloc[-len(df_prophet_test):].values
rezultate['Prophet'] = pred_prophet
print(f"Gata. Timp: {timpi['Prophet']:.1f}s")


# ======================================================
# MODEL 2: STACKED LSTM (Cerinta: 2-4 straturi, Dropout, Huber)
# ======================================================
print("\n--- 2. STACKED LSTM (Robust Loss) ---", flush=True)

model_lstm = Sequential()

# Strat 1: Return Sequences=True 
model_lstm.add(LSTM(64, return_sequences=True, input_shape=(WINDOW_SIZE, n_features)))
model_lstm.add(Dropout(0.3)) # Cerinta: Dropout 0.2-0.4

# Strat 2: Stacked
model_lstm.add(LSTM(32, return_sequences=False))
model_lstm.add(Dropout(0.3))

model_lstm.add(Dense(16, activation='relu'))
model_lstm.add(Dense(1)) # Output final

# Cerinta: Loss = Huber 
model_lstm.compile(optimizer='adam', loss=Huber(delta=1.0))

# Antrenare
start = time.time()
print("   -> Antrenare LSTM...", flush=True)
history_lstm = model_lstm.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)
timpi['LSTM'] = time.time() - start

# Predictie
pred_scaled = model_lstm.predict(X_test, verbose=0)
rezultate['LSTM'] = scaler.inverse_transform(pred_scaled).flatten()
print(f"Gata. Timp: {timpi['LSTM']:.1f}s")


# ======================================================
# MODEL 3: GRU (Cerinta: Comparatie cu LSTM)
# ======================================================
print("\n--- 3. GRU ---", flush=True)

model_gru = Sequential()
model_gru.add(GRU(64, return_sequences=True, input_shape=(WINDOW_SIZE, n_features)))
model_gru.add(Dropout(0.3))
model_gru.add(GRU(32, return_sequences=False))
model_gru.add(Dense(1))

# Folosim tot Huber 
model_gru.compile(optimizer='adam', loss=Huber(delta=1.0))

# Antrenare
start = time.time()
print("   -> Antrenare GRU...", flush=True)
model_gru.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)
timpi['GRU'] = time.time() - start

# Predictie
pred_scaled = model_gru.predict(X_test, verbose=0)
rezultate['GRU'] = scaler.inverse_transform(pred_scaled).flatten()
print(f"Gata. Timp: {timpi['GRU']:.1f}s")


# ======================================================
# MODEL 4: SIMPLE RNN (Cerinta: Gradient Clipping, LR mic)
# ======================================================
print("\n--- 4. SIMPLE RNN (Vanilla - Tuned) ---", flush=True)

model_rnn = Sequential()
model_rnn.add(SimpleRNN(32, input_shape=(WINDOW_SIZE, n_features), activation='tanh'))
model_rnn.add(Dropout(0.2))
model_rnn.add(Dense(1))

# Cerinta: Gradient Clipping & Learning Rate mic
# clipvalue=1.0 
opt = Adam(learning_rate=0.0001, clipvalue=1.0)

model_rnn.compile(optimizer=opt, loss='mse') 

# Antrenare
start = time.time()
print("   -> Antrenare RNN...", flush=True)
model_rnn.fit(X_train, y_train, epochs=5, batch_size=128, verbose=1)
timpi['RNN'] = time.time() - start

# Predictie
pred_scaled = model_rnn.predict(X_test, verbose=0)
rezultate['RNN'] = scaler.inverse_transform(pred_scaled).flatten()
print(f"Gata. Timp: {timpi['RNN']:.1f}s")


# ======================================================
# MODEL 5: DeepAR (Probabilistic cu Laplace Loss)
# ======================================================

# 1. Definim functia de Loss (Laplace Negative Log Likelihood)
def laplace_nll(y_true, y_pred):
    # Modelul scoate 2 valori: Mu (Prezicerea) si Sigma (Incertitudinea)
    mu = y_pred[:, 0]
    sigma_raw = y_pred[:, 1]

    # Sigma trebuie sa fie pozitiv, folosim softplus
    sigma = tf.nn.softplus(sigma_raw) + 1e-6

    # Formula matematica pentru Laplace: ln(2*sigma) + |y - mu| / sigma
    nll = tf.math.log(2.0 * sigma) + tf.abs(y_true - mu) / sigma
    return tf.reduce_mean(nll)

# 2. Construim Modelul
n_features = X_train.shape[2] 

model_deepar = Sequential()
model_deepar.add(LSTM(64, return_sequences=True, input_shape=(WINDOW_SIZE, n_features)))
model_deepar.add(Dropout(0.3))
model_deepar.add(LSTM(32, return_sequences=False))
model_deepar.add(Dropout(0.3))
model_deepar.add(Dense(16, activation="relu"))

# Stratul final are 2 neuroni: unul pentru valoare (Mu), unul pentru variatie (Sigma)
model_deepar.add(Dense(2))

# Compilam cu functia noastra custom
model_deepar.compile(optimizer='adam', loss=laplace_nll)

# --- START TIMER ---
print("   -> Antrenare DeepAR ...", flush=True)
start_time = time.time()

# Antrenam
model_deepar.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)
end_time = time.time()
# --- STOP TIMER ---

timpi['DeepAR'] = end_time - start_time

# Predictie
print("   -> Predictie DeepAR...", flush=True)
# Modelul returneaza o matrice cu 2 coloane: [Mu, Sigma]
pred_params = model_deepar.predict(X_test, verbose=0)

# Noi luam doar prima coloana (Mu) ca fiind predictia propriu-zisa
mu_pred_scaled = pred_params[:, 0]

# O transformam inapoi in Watti
pred_deepar = scaler.inverse_transform(mu_pred_scaled.reshape(-1, 1)).flatten()
rezultate['DeepAR'] = pred_deepar

print(f"Gata. Timp antrenare: {timpi['DeepAR']:.1f} s.")

# ======================================================
# MODEL 6: XGBoost & LightGBM (Feature Engineering)
# ======================================================
print("\n--- 6. XGBoost & LightGBM (Machine Learning Clasic) ---", flush=True)

# 1. Pregatire date TABELARE
df_ml = df_1min.copy()
target_col = 'Aggregate'

print("   -> Generare LAG-uri si Rolling Stats...", flush=True)
lags = [1, 5, 15, 60]
for lag in lags:
    df_ml[f'lag_{lag}'] = df_ml[target_col].shift(lag)

window_size = 60
df_ml['rolling_mean'] = df_ml[target_col].shift(1).rolling(window=window_size).mean()
df_ml['rolling_std'] = df_ml[target_col].shift(1).rolling(window=window_size).std()
df_ml.dropna(inplace=True)

features = [col for col in df_ml.columns if 'lag_' in col or 'rolling_' in col or col in ['Cluster', 'Hour_Sin', 'Hour_Cos', 'IsWeekend']]

X = df_ml[features].values
y = df_ml[target_col].values

# Split Train/Test (Respectand axa timpului)
test_len = len(test_data)
train_len = len(y) - test_len
X_train_ml = X[:train_len]
y_train_ml = y[:train_len]
X_test_ml = X[train_len:]
y_test_ml = y[train_len:]

# --- XGBOOST ---
print("\n   -> Antrenare XGBoost...", flush=True)
start = time.time()
model_xgb = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6, early_stopping_rounds=50, n_jobs=-1, random_state=42)
model_xgb.fit(X_train_ml, y_train_ml, eval_set=[(X_test_ml, y_test_ml)], verbose=0)
timp_xgb = time.time() - start

pred_xgb = model_xgb.predict(X_test_ml)
rezultate['XGBoost'] = pred_xgb
timpi['XGBoost'] = timp_xgb

# Calcul metrici XGBoost
mae_xgb = mean_absolute_error(y_test_ml, pred_xgb)
mse_xgb = mean_squared_error(y_test_ml, pred_xgb)
print(f"XGBoost GATA ({timp_xgb:.1f}s) | MAE: {mae_xgb:.2f} W | MSE: {mse_xgb:.2f}")

# --- LIGHTGBM ---
print("\n   -> Antrenare LightGBM...", flush=True)
start = time.time()
model_lgb = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=31, n_jobs=-1, random_state=42, verbosity=-1)
model_lgb.fit(X_train_ml, y_train_ml, eval_set=[(X_test_ml, y_test_ml)], eval_metric='mae', callbacks=[early_stopping(50), log_evaluation(0)])
timp_lgb = time.time() - start

pred_lgb = model_lgb.predict(X_test_ml)
rezultate['LightGBM'] = pred_lgb
timpi['LightGBM'] = timp_lgb

# Calcul metrici LightGBM
mae_lgb = mean_absolute_error(y_test_ml, pred_lgb)
mse_lgb = mean_squared_error(y_test_ml, pred_lgb)
print(f"LightGBM GATA ({timp_lgb:.1f}s) | MAE: {mae_lgb:.2f} W | MSE: {mse_lgb:.2f}")

# ======================================================
# SALVARE REZULTATE FINALE
# ======================================================
print("\nSalvare pachet rezultate...", flush=True)
results_package = {
    'rezultate_finale': rezultate,
    'test_data_index': test_data.index,
    'test_data_values': test_data['Aggregate'].values,
    'timpi_antrenare': timpi,
    'models_config': "LSTM, Prophet, RNN, GRU, DeepAR, XGBoost, LightGBM"
}

with open('model_results_final.pkl', 'wb') as f:
    pickle.dump(results_package, f)

print("TOTUL GATA! Rezultatele sunt in 'model_results_final.pkl. Poti rula 03_Analysis_Comparison.ipynb'", flush=True)

--- START MODEL TRAINING  ---
Incarcare date... GATA!
[INFO] Input Features: 6 (inclusiv Cluster pt regimuri)

--- 1. PROPHET (Tuned) ---
   -> Fit Prophet...


18:08:46 - cmdstanpy - INFO - Chain [1] start processing
18:17:58 - cmdstanpy - INFO - Chain [1] done processing


   -> Predictie Prophet...
Gata. Timp: 594.8s

--- 2. STACKED LSTM (Robust Loss) ---
   -> Antrenare LSTM...
Epoch 1/5
11501/11501 ━━━━━━━━━━━━━━━━━━━━ 403s 35ms/step - loss: 0.2144
Epoch 2/5
11501/11501 ━━━━━━━━━━━━━━━━━━━━ 429s 37ms/step - loss: 0.1800
Epoch 3/5
11501/11501 ━━━━━━━━━━━━━━━━━━━━ 431s 37ms/step - loss: 0.1706
Epoch 4/5
11501/11501 ━━━━━━━━━━━━━━━━━━━━ 433s 38ms/step - loss: 0.1660
Epoch 5/5
11501/11501 ━━━━━━━━━━━━━━━━━━━━ 434s 38ms/step - loss: 0.1614
Gata. Timp: 2131.6s

--- 3. GRU ---
   -> Antrenare GRU...
Epoch 1/5
11501/11501 ━━━━━━━━━━━━━━━━━━━━ 592s 51ms/step - loss: 0.2058
Epoch 2/5
11501/11501 ━━━━━━━━━━━━━━━━━━━━ 618s 54ms/step - loss: 0.1705
Epoch 3/5
11501/11501 ━━━━━━━━━━━━━━━━━━━━ 549s 48ms/step - loss: 0.1623
Epoch 4/5
11501/11501 ━━━━━━━━━━━━━━━━━━━━ 479s 42ms/step - loss: 0.1585
Epoch 5/5
11501/11501 ━━━━━━━━━━━━━━━━━━━━ 446s 39ms/step - loss: 0.1558
Gata. Timp: 2686.0s

--- 4. SIMPLE RNN (Vanilla - Tuned) ---
   -> Antrenare RNN...
Epoch 1/5
5751/575